In [59]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
import re
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [60]:
data = open('text.txt', 'r', encoding ='utf8').read()

In [61]:
print(data[:1000])

Алеся Казанцева

Режиссёр сказал: одевайся теплее, тут холодно



© Алеся Казанцева, 2019

© Владимир Сахнов, иллюстрации, 2019

© ООО «Издательство «Лайвбук», оформление, 2019




«Алеся Петровна обладает уникальным умением. Она берет ничтожнейшую частицу здешнего бытия – пыль, козявку, жука, – и раздувает эту неприметную частицу, не фиксируемую нашим обычным взглядом, сначала в значимый предмет, потом в очень значимый предмет, потом в сверхзначимый предмет, потом в перл бытия, и, наконец, в высшую форму существования: в текст. Если бы Алеся Петровна взялась за какой-нибудь мю-мезон, за какой-нибудь завалящий кварк, она построила бы из него дополнительную Вселенную, хорошо оборудованную, исполненную смысла, красивую и тревожную, насквозь пронизанную юмором. И мы бы все с удовольствием в нее переехали!»

    Татьяна Толстая, писательница



«За свою жизнь, я прочел три книги. Букварь, синюю и книгу Алеси. Про меня гениально, про Слепакова похуже, но там и масштаб личности не тот. А в ц

In [62]:
data = re.sub(r'[\d\t,.?!-]','', data).split('\n')
print(data[:10000])

['Алеся Казанцева', '', 'Режиссёр сказал: одевайся теплее тут холодно', '', '', '', '© Алеся Казанцева ', '', '© Владимир Сахнов иллюстрации ', '', '© ООО «Издательство «Лайвбук» оформление ', '', '', '', '', '«Алеся Петровна обладает уникальным умением Она берет ничтожнейшую частицу здешнего бытия – пыль козявку жука – и раздувает эту неприметную частицу не фиксируемую нашим обычным взглядом сначала в значимый предмет потом в очень значимый предмет потом в сверхзначимый предмет потом в перл бытия и наконец в высшую форму существования: в текст Если бы Алеся Петровна взялась за какойнибудь мюмезон за какойнибудь завалящий кварк она построила бы из него дополнительную Вселенную хорошо оборудованную исполненную смысла красивую и тревожную насквозь пронизанную юмором И мы бы все с удовольствием в нее переехали»', '', '    Татьяна Толстая писательница', '', '', '', '«За свою жизнь я прочел три книги Букварь синюю и книгу Алеси Про меня гениально про Слепакова похуже но там и масштаб личнос

In [63]:
def tokenize_data(data):
    tokens = [x.split() for x in data]
    return tokens

tokenized_data = tokenize_data(data)
print(tokenized_data)

[['Алеся', 'Казанцева'], [], ['Режиссёр', 'сказал:', 'одевайся', 'теплее', 'тут', 'холодно'], [], [], [], ['©', 'Алеся', 'Казанцева'], [], ['©', 'Владимир', 'Сахнов', 'иллюстрации'], [], ['©', 'ООО', '«Издательство', '«Лайвбук»', 'оформление'], [], [], [], [], ['«Алеся', 'Петровна', 'обладает', 'уникальным', 'умением', 'Она', 'берет', 'ничтожнейшую', 'частицу', 'здешнего', 'бытия', '–', 'пыль', 'козявку', 'жука', '–', 'и', 'раздувает', 'эту', 'неприметную', 'частицу', 'не', 'фиксируемую', 'нашим', 'обычным', 'взглядом', 'сначала', 'в', 'значимый', 'предмет', 'потом', 'в', 'очень', 'значимый', 'предмет', 'потом', 'в', 'сверхзначимый', 'предмет', 'потом', 'в', 'перл', 'бытия', 'и', 'наконец', 'в', 'высшую', 'форму', 'существования:', 'в', 'текст', 'Если', 'бы', 'Алеся', 'Петровна', 'взялась', 'за', 'какойнибудь', 'мюмезон', 'за', 'какойнибудь', 'завалящий', 'кварк', 'она', 'построила', 'бы', 'из', 'него', 'дополнительную', 'Вселенную', 'хорошо', 'оборудованную', 'исполненную', 'смысла', 

In [64]:
vocabulary = []
for sentence in tokenized_data:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [65]:
word2idx

{'Алеся': 0,
 'Казанцева': 1,
 'Режиссёр': 2,
 'сказал:': 3,
 'одевайся': 4,
 'теплее': 5,
 'тут': 6,
 'холодно': 7,
 '©': 8,
 'Владимир': 9,
 'Сахнов': 10,
 'иллюстрации': 11,
 'ООО': 12,
 '«Издательство': 13,
 '«Лайвбук»': 14,
 'оформление': 15,
 '«Алеся': 16,
 'Петровна': 17,
 'обладает': 18,
 'уникальным': 19,
 'умением': 20,
 'Она': 21,
 'берет': 22,
 'ничтожнейшую': 23,
 'частицу': 24,
 'здешнего': 25,
 'бытия': 26,
 '–': 27,
 'пыль': 28,
 'козявку': 29,
 'жука': 30,
 'и': 31,
 'раздувает': 32,
 'эту': 33,
 'неприметную': 34,
 'не': 35,
 'фиксируемую': 36,
 'нашим': 37,
 'обычным': 38,
 'взглядом': 39,
 'сначала': 40,
 'в': 41,
 'значимый': 42,
 'предмет': 43,
 'потом': 44,
 'очень': 45,
 'сверхзначимый': 46,
 'перл': 47,
 'наконец': 48,
 'высшую': 49,
 'форму': 50,
 'существования:': 51,
 'текст': 52,
 'Если': 53,
 'бы': 54,
 'взялась': 55,
 'за': 56,
 'какойнибудь': 57,
 'мюмезон': 58,
 'завалящий': 59,
 'кварк': 60,
 'она': 61,
 'построила': 62,
 'из': 63,
 'него': 64,
 'допол

In [66]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_data:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [67]:
idx_pairs[:1000]

array([[  0,   1],
       [  1,   0],
       [  2,   3],
       ...,
       [205, 204],
       [205,  31],
       [205, 206]])

In [68]:

def get_input_layer(words_idx):
    x = torch.zeros(words_idx.shape[0], vocabulary_size).float()
    x[list(range(0, words_idx.shape[0])), words_idx] = 1.
    return x


In [69]:
embedding_dims = 5

class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_line = nn.Linear(vocabulary_size, embedding_dims)
        self.output_line = nn.Linear(embedding_dims, vocabulary_size)
        
    def forward(self, X):
        X = F.relu(self.input_line(X))
        return F.softmax(self.output_line(X))


In [70]:

eta = 0.001
epochs = 1010
model = MyModule()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=eta)

In [71]:
losses = []
batch_size = 59
for epoch in range(epochs):
  loss_val = []
  for batch in range(0,idx_pairs.shape[0], batch_size):
    x = get_input_layer(idx_pairs[batch:batch+batch_size, 0])
    target = torch.from_numpy(np.array([idx_pairs[batch:batch+batch_size, 1]]))

    y = model(x)
    loss = loss_function(y, target.view(-1))
    loss_val.append(loss.item())
    loss.backward()
    optimizer.step()

    model.zero_grad()

  losses.append(np.mean(loss_val))
  if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {np.mean(loss_val)}')


<ipython-input-69-02e0ab43648f>:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.output_line(X))


RuntimeError: expected scalar type Long but found Int